In [6]:
import numpy as np
import pandas as pd
from pymongo import MongoClient
import glob
import datetime
import traceback

## MongoDB

In [2]:
client = MongoClient("mongodb://localhost:27017/")
db = client["local"]
presidencial = db["mesas-presidencial"]
distrital = db["mesas-distrital"]
dip_nacional = db["mesas-nacional"]
municipal = db["mesas-alcaldias"]

In [ ]:
def limpiar_mongo(collections):
    for col in collections:
        print(f'eliminados {col.delete_many({}).deleted_count} documentos de {col.name}')

In [332]:
collections = [presidencial, dip_nacional, distrital, municipal]
limpiar_mongo(collections)

eliminados 13407 documentos de mesas-presidencial
eliminados 10466 documentos de mesas-nacional
eliminados 10307 documentos de mesas-distrital
eliminados 10187 documentos de mesas-alcaldias


## Leer Datos TSE

In [3]:
def leer_csv_tse(path):
    df = pd.read_csv(path,  sep=',', header=4, index_col=0)
    df = df.drop(columns=['OBSERVACIONES', 'CÓDIGO_INTEGRIDAD', 'FECHA_HORA_PUBLICACIÓN', 'DIGITALIZACIÓN', 'ID_DEPARTAMENTO', 'ID_MUNICIPIO', 'PADRÓN'])
    df['ARCHIVO'] = path
    df['FECHA_ARCHIVO'] = pd.read_csv(path, nrows=1, skiprows=[0], header=None).iloc[0,0]
    return df

def ingerir_directorio_tse(path, collection):
    csv_paths = glob.glob(path + r'\*.csv')
    result = []
    for csv_path in csv_paths:
        no_contabilizadas = pd.NA
        insertadas = 0
        try:
            data = leer_csv_tse(csv_path)
            no_contabilizadas = cuenta_no_contabilizadas(data)
            data = solo_contabilizadas(data)
            if data.empty:
                print(f'archivo no contiene actas contabilizadas {csv_path}')
            else:    
                insert_result = collection.insert_many(convertir_a_mongo_docs(data))
                insertadas = len(insert_result.inserted_ids)
        except Exception as e:
                print(f"archivo: {csv_path} error: {str(e)}")
        file_result = {'archivo' : csv_path, ' no_contabilizadas' : no_contabilizadas, 'insertadas' : insertadas}
        result.append(file_result)
    return pd.DataFrame(result)

def llenar_nans(data):
    return data.applymap(lambda l: l if not pd.isna(l) else np.random.choice(100))

def solo_contabilizadas(df):
    return df[df['CONTABILIZADA'] == 1]

def cuenta_no_contabilizadas(df):
    return len(df[df['CONTABILIZADA'] != 1])

def convertir_a_mongo_docs(data):
    data_dict = data.transpose().to_dict()
    for mesa in data_dict:
        data_dict[mesa]['mesa'] = mesa
    return list(data_dict.values())

## Leer Datos Fiscales

In [4]:
def leer_excel_fiscales(path):
    xlsx = pd.ExcelFile(path)
    hojas_mesas = xlsx.sheet_names[1:-1]
    xlsx = pd.read_excel(path, header=5, index_col=1, sheet_name=hojas_mesas)
    dataframes = {}
    for hoja in xlsx:
        #print(path, hoja)
        df = xlsx[hoja]
        if df.empty:
            continue
        df = df.drop(df.columns[0], axis=1).transpose()
        #df.columns = df.columns.str.strip()
        nan_header_column_index = df.columns.tolist().index(np.nan)
        df = df.iloc[:, :nan_header_column_index]
        if not df.empty:
            dataframes[hoja]=df
    return dataframes    
    
#list(leer_excel_fiscales('excel-pruebas\Pruebas-AV.xlsx').values())[0]

In [9]:
def nombres_columnas(directorio):
    xlsx_paths = glob.glob(directorio + r'\**\*.xlsx', recursive=True)
    result = set()
    
    for path in xlsx_paths:
        try:
            hojas = leer_excel_fiscales(path)
            for hoja in hojas.keys():
                nombres = hojas[hoja].columns.to_list()
                result.update(nombres)
        except Exception as e:
            print(path, e)
            traceback.print_exc()
            continue
   
    return sorted(result)


print(datetime.datetime.now().time())
nombres = nombres_columnas(r'C:\Users\batoz\Downloads\Elecciones 2023-20230629-00')
print(datetime.datetime.now().time())
nombres

10:01:08.191224
C:\Users\batoz\Downloads\Elecciones 2023-20230629-00\Elecciones 2023\Presidencial\Solola(1).xlsx nan is not in list


Traceback (most recent call last):
  File "C:\Users\batoz\AppData\Local\Temp\ipykernel_7464\1135238011.py", line 7, in nombres_columnas
    hojas = leer_excel_fiscales(path)
  File "C:\Users\batoz\AppData\Local\Temp\ipykernel_7464\2852799679.py", line 13, in leer_excel_fiscales
    nan_header_column_index = df.columns.tolist().index(np.nan)
ValueError: nan is not in list


10:01:52.265464


[' BLA NCO',
 ' NULOS',
 ' PARTIDO AZUL (AZUL)',
 ' TOTAL DE VOTOS VALIDAMENTE EMITIDOS',
 ' VALIDOS ',
 ' nulos',
 ' vos',
 '(Bien',
 '(une)',
 '(vamos)',
 '(victoria',
 'AZUL',
 'AZUL ',
 'Azul',
 'BICION CON VALORES (VIVA)',
 'BIEN',
 'BIEN ',
 'BIENESTAR',
 'BIENESTAR NACIONA (BIEN)',
 'BIENESTAR NACIONAL',
 'BIENESTAR NACIONAL ',
 'BIENESTAR NACIONAL (BIEN ',
 'BIENESTAR NACIONAL (BIEN)',
 'BIENESTAR NACIONAL (BIM)',
 'BIENESTAR NACIONAL(BIEN)',
 'BIENESTR NACIONAL  ',
 'BLANCO',
 'BLANCOS',
 'BOLUNTAD , OPORTUNIDAD Y SOLIRIDAD (VOS)',
 'Bien',
 'Bienestar Nacional (BIEN)',
 'Bienestar Nacional (Bien)',
 'Bienestar Nacional Bien',
 'Bienestar nacional',
 'Blanco',
 'CABAL',
 'CABAL ',
 'CABAL  (CABAL)',
 'CABAL (CABAL',
 'CABAL (CABAL )',
 'CABAL (CABAL)',
 'CABAL (CABAL) ',
 'CABAL(CABAL)',
 'CAMBIO',
 'CAMBIO ',
 'CAMBIO  (CAMBIO)',
 'CAMBIO (CAMBIO)',
 'CAMBIO(CAMBIO)',
 'CAMIO',
 'CAVAL(CAVAL)',
 'COCIG',
 'COMITE CHERIFON',
 'COMITE CIVICO JUSTICIA',
 'COMITE CIVICO TRABAJAND

## Comparar

In [5]:
def find_mesas(mesa_inicio, mesa_fin, collection):
    query = {"mesa": {"$gte": mesa_inicio, "$lte": mesa_fin}}    
    result = {}
    for datos_mesa in collection.find(query):
        result[datos_mesa['mesa']] = datos_mesa
    return result

def to_int_or_nan(value, key):
    vaue = value.strip() if isinstance(value, str) else value
    if not pd.isna(value) and isinstance(value, str) and not value.isnumeric():
        print(f'WARN - found value {value} for key {key}')
        return np.NaN
    return int(value) if not pd.isna(value) else np.NaN

def comparar_mesa(no_mesa, datos_mesa_fiscal, datos_mesa_tse, path):
    result = []
    mismatched_cols = []
    for key in datos_mesa_fiscal.keys():
        if key in datos_mesa_tse:
            valor_fiscal = to_int_or_nan(datos_mesa_fiscal[key], key)
            valor_tse = to_int_or_nan(datos_mesa_tse[key], key)
            if not np.isnan(valor_fiscal) and valor_fiscal != valor_tse:
                result.append({
                    'mesa' : no_mesa,
                    'columna' : key,
                    'tse': valor_tse,
                    'fiscal': valor_fiscal,
                    'dif' :np.nan_to_num(np.subtract(valor_tse, valor_fiscal))
                })
        else:
            mismatched_cols.append(key)
    return (result, mismatched_cols)
            

def comparar_archivo(path, mongo_collection):
    result = []
    mismatched_cols = {}
    hojas = leer_excel_fiscales(path)
    for hoja in hojas.keys():
        mesas_excel = hojas[hoja]
        mesa_inicio = mesas_excel.iloc[0].name
        mesa_fin = mesas_excel.iloc[-1].name
        mesas_db = find_mesas(mesa_inicio, mesa_fin, mongo_collection)
        for no_mesa, mesa_fiscal in mesas_excel.iterrows():
            if no_mesa in mesas_db:
                mesa_fiscal = mesa_fiscal.to_dict()
                mesa_tse = mesas_db[no_mesa]
                (diferencias, mismatches) = comparar_mesa(no_mesa, mesa_fiscal, mesa_tse, path)
                result.extend(diferencias)
                mismatched_cols[hoja] = mismatches
    return (pd.DataFrame(result), mismatched_cols)

pd.set_option('display.max_rows', 1000)
comparar_archivo('excel-pruebas\Chimaltenango.xlsx', distrital)[0][lambda x: x['mesa'] == 6785]


#hojas = leer_excel_fiscales('excel-pruebas\Chimaltenango.xlsx')['EL TEJAR']
#mesa_tse = hojas.loc[6794]
#mesa_db = find_mesas(6794, 6794, distrital)
#comparar_mesa(6794, mesa_db, mesa_tse, '')

WARN - found value -- for key FCN-NACION
WARN - found value -- for key TODOS
WARN - found value -- for key PHG
WARN - found value -- for key PIN
WARN - found value -- for key MLP


,mesa,columna,tse,fiscal,dif
24,6785,UNIONISTA,3,4,-1
25,6785,CABAL,24,28,-4
26,6785,AZUL,11,6,5
27,6785,VAMOS,46,32,14
28,6785,PHG,0,1,-1
29,6785,VICTORIA,5,8,-3
30,6785,MLP,2,0,2
31,6785,UR,1,2,-1
32,6785,PIN,0,1,-1
33,6785,BIEN,3,6,-3


In [ ]:
def comparar_todos_los_archivos(directorio, collection):
    xlsx_paths = glob.glob(directorio + r'\*.xlsx')
    result = pd.DataFrame()
    mismatches = {}
    for path in xlsx_paths:
        (diffs, col_mismatches) = comparar_archivo(path, collection)
        pd.concat([result, diffs])
        mismatches[path] = col_mismatches
    return (result, mismatches)

(result, mismatches) = comparar_todos_los_archivos(r'datos-fiscales\manuales\presidencial', presidencial)
(mismatches, result)


In [ ]:
(result, mismatches) = comparar_todos_los_archivos(r'datos-fiscales\manuales\distrital', distrital)
(mismatches, result)

In [ ]:
(result, mismatches) = comparar_todos_los_archivos(r'datos-fiscales\manuales\nacional', dip_nacional)
(mismatches, result)

## Llenar base de datos

In [336]:
limpiar_mongo([presidencial])
ingerir_directorio_tse(r'datos-tse\reales\1-PRESIDENTE', presidencial)

eliminados 13152 documentos de mesas-presidencial


,archivo,no_contabilizadas,insertadas
0,datos-tse\reales\1-PRESIDENTE\gtm2023_e1.csv,289,24296


In [337]:
limpiar_mongo([dip_nacional])
ingerir_directorio_tse(r'datos-tse\reales\2-DIP-NAC', dip_nacional)

eliminados 10155 documentos de mesas-nacional


,archivo,no_contabilizadas,insertadas
0,datos-tse\reales\2-DIP-NAC\gtm2023_e2.csv,324,24103


In [338]:
limpiar_mongo([distrital])
ingerir_directorio_tse(r'datos-tse\reales\3-DIP-DIST', distrital)

eliminados 0 documentos de mesas-distrital


,archivo,no_contabilizadas,insertadas
0,datos-tse\reales\3-DIP-DIST\gtm2023_e3d00.csv,13,2010
1,datos-tse\reales\3-DIP-DIST\gtm2023_e3d01.csv,71,3242
2,datos-tse\reales\3-DIP-DIST\gtm2023_e3d02.csv,1,550
3,datos-tse\reales\3-DIP-DIST\gtm2023_e3d03.csv,48,874
4,datos-tse\reales\3-DIP-DIST\gtm2023_e3d04.csv,0,346
5,datos-tse\reales\3-DIP-DIST\gtm2023_e3d05.csv,14,1164
6,datos-tse\reales\3-DIP-DIST\gtm2023_e3d06.csv,5,704
7,datos-tse\reales\3-DIP-DIST\gtm2023_e3d07.csv,2,704
8,datos-tse\reales\3-DIP-DIST\gtm2023_e3d08.csv,3,655
9,datos-tse\reales\3-DIP-DIST\gtm2023_e3d09.csv,2,1318


In [339]:
limpiar_mongo([municipal])
ingerir_directorio_tse(r'datos-tse\reales\4-CORPORACIÓN', municipal)

eliminados 9868 documentos de mesas-alcaldias
archivo no contiene actas contabilizadas datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d01m04.csv
archivo no contiene actas contabilizadas datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d11m04.csv


,archivo,no_contabilizadas,insertadas
0,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d00m0...,8,2015
1,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d01m0...,0,151
2,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d01m0...,0,122
3,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d01m0...,24,0
4,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d01m0...,0,102
...,...,...,...
335,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d22m1...,0,41
336,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d22m1...,0,72
337,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d22m1...,0,24
338,datos-tse\reales\4-CORPORACIÓN\gtm2023_e4d22m1...,0,26
